# Fashion Advisor User Interface

We create a user interface that is sequential in nature:
- Step 1: The users upload the picture and confirm their choice
- Step 2: The clothes parsing YOLO result is displayed and requires user confirmation
- Step 3: Score is displayed

So far, we have created a user interface that would work up until Step 2.
We note that this code requires the referencing of the "Fashion_Advisor_GUI" folder to run

In [17]:
# Libraries for the user interface
import os
from tkinter import *
from tkinter import filedialog
!pip install Pillow
from PIL import ImageTk, Image

# Libraries for YOLO clothes parsing
import matplotlib.pyplot as plt
!pip install ultralytics
from ultralytics import YOLO
import cv2
import numpy as np

In [18]:
# Creating window
root = Tk()

# Window title
root.title("Fashion Advisor")

# Window Icon
icon_path = os.path.join('Fashion_Advisor', 'logo.ico')
root.iconbitmap(icon_path)

# Setting window size
root.geometry("500x520")

# Setting Background  
bg = ImageTk.PhotoImage(Image.open(os.path.join('Fashion_Advisor', 'bg.jpg'))) # bg size = 1078 x 720
my_bg = Label(root, image = bg)
my_bg.place(x = 0, y = 0, relwidth = 1, relheight = 1)

In [19]:
# Function including all the step 1 widgets and related functions
def step1():
    # Disabling step 2 in case someone reached step 2 and reclicked step 1 (to restart the process)
    step2_btn.config(state = 'disabled')
    
    # Setting background
    bg2 = ImageTk.PhotoImage(Image.open(os.path.join('Fashion_Advisor', 'bg.jpg')))
    my_bg2 = Label(main_frame, image = bg)
    my_bg2.place(x = 0, y = 0, relwidth = 1, relheight = 1)
    
    # Inserting welcome text
    welcome_label = Label(main_frame, text = "Your Fashion Advisor is here to help!", font = ("Bold", 10), bg = "white")
    welcome_label.place(relx = 0.5, y = 10, anchor = CENTER)
    
    # Guide text
    guide_label = Label(main_frame, text = "Show me your outfit!", font = ("Bold", 10), bg = "white")
    guide_label.place(relx = 0.5, y = 70, anchor = CENTER)
    
    # Picture Frame
    pic_frame = LabelFrame(main_frame, width = 100, height = 100, bg = "white") 
    pic_frame.place(relx = 0.5, y = 200 , anchor = CENTER) 
    
    # Function to resize an image while respecting its aspect ratio
    def resize_img(input_img, max_size):
        global resized_img
        width, height = input_img.size
        
        # Determine the aspect ratio
        aspect_ratio = width/height
        
        # Calculate the new dimensions: 
        if aspect_ratio > 1:
            new_width = max_size
            new_height = int(max_size / aspect_ratio)
        else:
            new_width = int(max_size * aspect_ratio)
            new_height = max_size
        
        # Resize the image while maintaining the aspect ratio
        resized_img = input_img.resize((new_width, new_height), Image.LANCZOS)
        return resized_img
        
    # Function for file selection and upload
    def select_file():
        global img, img_directory

        # Opening the select window file 
        root.filename = filedialog.askopenfilename(initialdir = os.path.join('C:\\', 'Users', 'Carine', 'Desktop',\
                                                                             'Spring_2023', 'Project', 'GUI'), \
                                                   title = "Select a file", \
                                                   filetypes = (("All", "*.*"),("PNG files", "*.png"),("JPEG files", "*.jpg"),("JPEG files", "*.jpeg"), \
                                                                ("JFIF files", "*.jfif"),("TIF files","*.tif")))

        # We add an if statement to handle the case where the user does not select any file and closes the window
        if root.filename:
            img_directory = root.filename

            # Enabling the upload button
            upload_button.config(state = "normal")

            # Deleting the select button
            select_button.pack_forget()

            # Displaying the image inside the frame
            img = Image.open(img_directory)
            #resized_img = img.resize((200,200), Image.LANCZOS)  # Redefine the way the images are being resized
            img = resize_img(img, 200)
            img = ImageTk.PhotoImage(img)
            img_label = Label(pic_frame, image = img)
            img_label.pack()

            # Change the guide sentence
            guide_label = Label(main_frame, text = "Click upload if this is the outfit you wish to evaluate", bg = "white")
            guide_label.place(relx = 0.5, y = 70, anchor = CENTER)
        else:
            return
                        
    def upload_file(directory):
        global dir_yolo, outfit 
        # Reading the image for YOLO clothes parsing
        dir_yolo = directory
        outfit = plt.imread(directory)
        
        # Guide label to go to step 2
        ready_label = Label(main_frame, text = "You are ready for step 2!", font = ("bold", 10), bg = "white")
        ready_label.place(relx = 0.5, y = 480,  anchor = CENTER)
        
        # Enabling step 2 button
        step2_btn.config(state = "normal")
        step2_btn.invoke()

    # Button that allows the file selection
    global select_img
    select_img = Image.open(os.path.join('Fashion_Advisor', 'select4.png')) # Change picture
    select_img = select_img.resize((200,220), Image.LANCZOS)
    select_img = ImageTk.PhotoImage(select_img)
    select_button = Button(pic_frame, image = select_img, bg = "white", padx = 100,  pady = 100, command = select_file)
    select_button.pack()

    # Button that allows reading the image
    upload_button = Button(main_frame, text = "Upload", padx = 50, bg = "red", fg = "white", \
                       command = lambda: upload_file(img_directory), state = DISABLED)
    upload_button.place(relx = 0.5, y = 330, anchor = CENTER)

In [20]:
def step2():

    # Setting background
    bg2 = ImageTk.PhotoImage(Image.open(os.path.join('Fashion_Advisor', 'bg.jpg')))
    my_bg2 = Label(main_frame, image = bg)
    my_bg2.place(x = 0, y = 0, relwidth = 1, relheight = 1)
    
    # Inserting welcome text
    welcome_label = Label(main_frame, text = "Your Fashion Advisor is here to help!", font = ("Bold", 10), bg = "white")
    welcome_label.place(relx = 0.5, y = 10, anchor = CENTER)
    
    # Guide text
    guide_label = Label(main_frame, text = "Are these all the items in your outfit?", font = ("Bold", 10), bg = "white")
    guide_label.place(relx = 0.5, y = 50, anchor = CENTER)
    
    # Function to resize an image while respecting its aspect ratio
    def resize_img(input_img, max_size):
        global resized_img
        width, height = input_img.size
        
        # Determine the aspect ratio
        aspect_ratio = width/height
        
        # Calculate the new dimensions: 
        if aspect_ratio > 1:
            new_width = max_size
            new_height = int(max_size / aspect_ratio)
        else:
            new_width = int(max_size * aspect_ratio)
            new_height = max_size
        
        # Resize the image while maintaining the aspect ratio
        resized_img = input_img.resize((new_width, new_height), Image.LANCZOS)
        return resized_img
        
    # Clothes parsing using YOLO
    model = YOLO(os.path.join("Fashion_Advisor","YOLO","detect","train","weights","best.pt"))
    class_names = ["Sunglasses","hat","jacket","shirt","pants", "shorts","skirt","dress", "bag", "shoes"]
    
    outfit_prediction = model.predict(source = dir_yolo, conf = 0.4, save = True, line_thickness = 2)
    boxes = outfit_prediction[0].boxes
    
    cropped_images = {}
    for i, box in enumerate(boxes.xyxy):
        class_id = boxes.cls[i]
        x1, y1, x2, y2 = box
        x1 = round(x1.item())
        y1 = round(y1.item())
        x2 = round(x2.item())
        y2 = round(y2.item())

        # Get the class name from the class ID
        class_name = class_names[int(class_id.item())]

        # Crop the image based on the bounding box
        crop_img = outfit[y1:y2, x1:x2]

        # Store the cropped image as a variable in the dictionary
        cropped_images[f"{class_name}"] = crop_img
    
    # Displaying the cropped garments inside sub-frames
    # Define the size of the sub-frames
    sub_width = 130
    sub_height = 130
    
    # Getting the size of the main frame
    main_width = main_frame.winfo_width()  
    
    # Calculating the number of sub-frames that can fit horizontally 
    cols = main_width // sub_width
    
    # Create the center-frame to position the sub-frames in the center
    center_frame = Frame(main_frame, bg = "white")
    center_frame.pack()
    
    # Creating the sub-frames and display the images
    for i, (name, img) in enumerate(cropped_images.items()):
        
        # Converting the image to a Tkinter-compatible format and resize the image to fit the sub-frame
        img = Image.fromarray(img)
        # resized_img = img.resize((110,110), Image.LANCZOS)  # Redefine the way the images are being resized
        img = resize_img(img,110)
        # Create a square image to maintain the square size of the label frame
        square_img = Image.new("RGB", (110, 110))
        offset_x = (110 - img.width) // 2
        offset_y = (110 - img.height) // 2
        square_img.paste(img, (offset_x, offset_y))
        garment = ImageTk.PhotoImage(square_img)

        # Creating the sub-frame
        col = i % cols
        row = i // cols
        sub_frame = LabelFrame(center_frame,text = name, width = sub_width, height = sub_height, padx=5, pady=5, bg = "white")
        sub_frame.grid(row = row, column = col)
        
        # Displaying the image in the sub-frame
        garment_label = Label(sub_frame, image = garment)
        garment_label.image = garment
        garment_label.pack()
    
    # Update the main frame to get the width and height of the center-frame
    main_frame.update() 
    
    # Calculate the x position to center the center-frame in the main frame
    x_center = (main_width - center_frame.winfo_width()) // 2

    # Place the center-frame in the main frame
    center_frame.place(x = x_center, y = 60)

In [21]:
# Inserting steps bar and main frame:
# Defining main frame and setting background

main_frame = Frame(root)
main_frame.pack(side = TOP)
main_frame.pack_propagate(False)
main_frame.configure(width = 500, height = 490)

bg2 = ImageTk.PhotoImage(Image.open(os.path.join('Fashion_Advisor', 'bg.jpg')))
my_bg2 = Label(main_frame, image = bg)
my_bg2.place(x = 0, y = 0, relwidth = 1, relheight = 1)

steps_frame = Frame(root, bg = "white")
steps_frame.pack(side = BOTTOM)
steps_frame.pack_propagate(False)
steps_frame.configure(width = 500, height = 30)

def hide_indicators():
    step1_ind.config(bg = "white")
    step2_ind.config(bg = "white")
    
def delete_page():
    for frame in main_frame.winfo_children():
        frame.destroy()
    
def indicate(selected_btn, step):
    hide_indicators()
    selected_btn.config(bg = "red")
    delete_page()
    step()

# Buttons that would allow us to navigate through the steps
step1_btn = Button(steps_frame, text = "Step 1", font = ("Bold",10), \
                   fg = "red", bg = "white", bd = 0, command = lambda: indicate(step1_ind, step1))
step2_btn = Button(steps_frame, text = "Step 2", font = ("Bold",10), \
                   fg = "red", bg = "white", bd = 0, command = lambda: indicate(step2_ind, step2), state = DISABLED)

step1_btn.place(relx = 0.8, y = 0)
step2_btn.place(relx = 0.9, y = 0)

# Indicators that will tell us at which state we are
step1_ind = Label(steps_frame, text = "", bg = "red")
step2_ind = Label(steps_frame, text = "", bg = "white")

step1_ind.place(relx = 0.8, y = 25, width = 45, height = 5)
step2_ind.place(relx = 0.9, y = 25, width = 45, height = 5)

# Set the step 1 button to be already clicked by default
step1_btn.invoke()

'None'

In [22]:
# Running the program
root.mainloop()


image 1/1 C:\Users\Carine\Desktop\Spring_2023\Project\GUI\test_pictures\Peroulla.jpeg: 640x480 1 hat, 1 shirt, 1 pants, 1 shoe, 437.0ms
Speed: 18.6ms preprocess, 437.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict49
